# Pytorch QuickStart

https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
trainingData = datasets.FashionMNIST(
    root='./data',
    train=True,
    download=True,
    transform=ToTensor(),
)

testData = datasets.FashionMNIST(
    root='data',
    train=False,
    download=False,
    transform=ToTensor()
)

In [3]:
batch_size = 64

trainDataLoader = DataLoader(trainingData, batch_size)
testDataLoader = DataLoader(testData, batch_size)

for X, y in trainDataLoader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape}, {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]), torch.int64


In [4]:
device = ("cuda" if torch.cuda.is_available()
         else "mpu" if torch.backends.mps.is_available()
         else "cpu")
print(device)

cpu


In [11]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512,10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [12]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [14]:
def train(dataLoader, model, loss_fn, optimizer):
    size = len(dataLoader.dataset)
    model.train()
    for batch, (X,y) in enumerate(dataLoader):
        X,y = X.to(device), y.to(device)
        
        # Compute prediction errors
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropogation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), (batch+1)*len(X)
            print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [15]:
def test(dataLoder, model, loss_fn):
    size = len(dataLoder.dataset)
    num_batches = len(dataLoder)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataLoder:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \nAccuracy: {100*correct:>0.1f}%, Avg Loss: {test_loss:>8f}\n")

In [16]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n------------")
    train(trainDataLoader, model, loss_fn, optimizer)
    test(testDataLoader, model, loss_fn)
print("Done!")

Epoch 1
------------
loss: 2.299843 [   64/60000]
loss: 2.292382 [ 6464/60000]
loss: 2.279303 [12864/60000]
loss: 2.277548 [19264/60000]
loss: 2.253531 [25664/60000]
loss: 2.231039 [32064/60000]
loss: 2.231089 [38464/60000]
loss: 2.200249 [44864/60000]
loss: 2.198424 [51264/60000]
loss: 2.175351 [57664/60000]
Test Error: 
Accuracy: 46.7%, Avg Loss: 2.164085

Epoch 2
------------
loss: 2.162625 [   64/60000]
loss: 2.155044 [ 6464/60000]
loss: 2.108943 [12864/60000]
loss: 2.129638 [19264/60000]
loss: 2.068275 [25664/60000]
loss: 2.024422 [32064/60000]
loss: 2.042187 [38464/60000]
loss: 1.971212 [44864/60000]
loss: 1.975493 [51264/60000]
loss: 1.908477 [57664/60000]
Test Error: 
Accuracy: 58.7%, Avg Loss: 1.901243

Epoch 3
------------
loss: 1.922940 [   64/60000]
loss: 1.892812 [ 6464/60000]
loss: 1.790696 [12864/60000]
loss: 1.833462 [19264/60000]
loss: 1.714584 [25664/60000]
loss: 1.674090 [32064/60000]
loss: 1.692048 [38464/60000]
loss: 1.597972 [44864/60000]
loss: 1.618531 [51264/600

In [17]:
torch.save(model.state_dict(), 'model.pth')
print("Saved PyTorch model state to model.pth")

Saved PyTorch model state to model.pth


In [18]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load('model.pth'))

/tmp/ipykernel_1508/729305697.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth'))


<All keys matched successfully>

In [22]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
X, y = testData[0][0], testData[0][1]
with torch.no_grad():
    X = X.to(device)
    pred = model(X)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f"Predicted: {predicted}, Actual: {actual}")

Predicted: Ankle boot, Actual: Ankle boot
